<a href="https://colab.research.google.com/github/uqglmn/dhv/blob/vi/22031854.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # Assignment - Pandas

# ## Read Libraries

get_ipython().system('pip install wbgapi')

import wbgapi as wb, pandas as pd, numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Country Selection Details
allcntry = [['ITA', 'NOR', 'SGP', 'COL', 'NCL', 'TUR', 'YUG', 'AND', 'MMR','LTU','CYP'],
      ['Italy', 'Norway', 'Singapore','Colombia', 'New Caledonia', 
       'Turkey', 'Tonga' , 'Andorra', 'Myanmar', 'Lithuania',"Cyprus"],
     [['Europe', 'Europe', 'Asia', 'South America', 'Oceania', 'Asia', 'Oceania', 'Europe', 'Asia', 'Europe',"Asia"]]]
 
# Dictionary with country code and names
dctcntr = {
    allcntry[0][i]:allcntry[1][i]
    for i in range(len(allcntry[0]))
}

# Dictionary with country code and groups
dctgrp = {
    allcntry[0][i]:allcntry[2][i]
    for i in range(len(allcntry[2]))
}

# ### Data Reading

first_indicator = ["EN.URB.LCTY", "SP.POP.1564.FE.IN", "SP.POP.1564.MA.IN", "SP.POP.65UP.MA.IN"]
fid_names = ["Population in largest city", "Population ages 15-64, female", 
             "Population ages 15-64, male", "Population ages 65 and above, male"]

second_indicator = ["NY.GDP.MKTP.CD", "NY.GDP.MKTP.KD.ZG", "NY.GDP.PCAP.KD", "NY.GDP.DEFL.KD.ZG.AD"]
sidnames = ["GDP Total", "GDP growth", "GDP per capita", "Inflation"]

# Read data for indicator-1 with lates 10 years
popdf  = wb.data.DataFrame(first_indicator, allcntry[0], mrv = 10) 

# Read data for indicator-2 with lates 10 years  
gdpdf  = wb.data.DataFrame(second_indicator, allcntry[0], mrv = 10) 
  
print("Collected Data from World Bank")
actdata = pd.concat([popdf, gdpdf])
actdata.to_csv("https://raw.githubusercontent.com/uqglmn/dhv/main/vis/Population_GDP_10Y_2012_2020.csv")
display(HTML(actdata.to_html()))
popix = popdf.index.tolist()
fdata,sdata = [],[]
popgdpnms = fid_names+sidnames
popgdpixs = first_indicator+second_indicator
for i in range(len(popix)):
    fdata.append(popix[i][0])
    sdata.append(popix[i][1])
idx_mlt = []

# creating multi-index data
for i in range(len(sdata)):
    idx_mlt.append(popgdpnms[popgdpixs.index(sdata[i])])     
popdf.index = idx_mlt

# taking the indices
gdpix = gdpdf.index.tolist()      
fdata,sdata = [],[]
popgdpnms = fid_names+sidnames
popgdpixs = first_indicator+second_indicator
for i in range(len(gdpix)):
    fdata.append(gdpix[i][0])
    sdata.append(gdpix[i][1])
idx_mlt = []

# creating data with multi-level indices
for i in range(len(sdata)):
    idx_mlt.append(popgdpnms[popgdpixs.index(sdata[i])])    
gdpdf.index = idx_mlt
ln = len(first_indicator)
init_val = 0
dataall = []
for c in range(len(allcntry[0])):
    try:
        df = pd.concat([popdf.iloc[init_val:ln].T,gdpdf.iloc[init_val:ln].T], axis = 1)
        idx_row = []
        for dfi in range(len(df.index)):
            idx_row.append(tuple([allcntry[0][c],df.index[dfi][2:]]))
            
        # Creating multi-leevl-index for dataframe
        mlid = pd.MultiIndex.from_tuples(idx_row)       
        ftid = []
        for dfc in range(len(df.columns)):
            
            # creating columns parameters by Education
            if dfc<4:
                ftid.append(tuple(["Population", df.columns[dfc]]))  
                
             # creating columns parameters by Health
            else:
                ftid.append(tuple(["Economy", df.columns[dfc]]))  
                
         # Creating multilevel column
        mlcol = pd.MultiIndex.from_tuples(ftid)    

         # Creating multilevel-index dataframe
        df = pd.DataFrame(df.values, columns = mlcol, index = mlid)   
        dataall.append(df)
        init_val = ln
        ln = ln+4
    except:
        pass
PopGDP = pd.concat(dataall)
print(len(PopGDP))
PopGDP

# Getting data information to check for missing values
PopGDP.info()    

# Replacing missing data by median
PopGDP = PopGDP.fillna(PopGDP.mean())   
PopGDP.isna().sum()
PopGDP.info()

# Country names
all_countries = []    
st = 2012
yrs = ["2012", "2013", "2014", "2015", "2016", "2018", "2019", "2020", "2021", "2017"]
years = []
for i in range(10):
    years.extend(yrs)
year = []
for i in range(len(allcntry[1])-1):
    for x in range(10):
        all_countries.append(str(allcntry[1][i]))
# Inserting country names into data feature
print(len(years))
PopGDP.head()

PopGDP['Country'] = all_countries
PopGDP['Years'] = years
PopGDP = PopGDP.drop(PopGDP.columns.tolist()[-3],axis=1)
print(PopGDP.columns)
PopGDP.head()

cols = []
for i in PopGDP.columns:
    cols.append(i[1])
print(cols)
PopGDP = pd.DataFrame(PopGDP.values,columns = cols)
cls = PopGDP.columns.tolist()
cls[-2] = "Country"
cls[-1] = "Year"
PopGDP.columns = cls
for i in PopGDP.columns.tolist()[:-2]:
    PopGDP[i] = np.array(PopGDP[i], float)

PopGDP.head()

fig = plt.figure(figsize=(14,10))
gs = fig.add_gridspec(nrows=2, ncols=3,height_ratios=[1,1])
PopGDP0=PopGDP.groupby('Year').agg({"Population in largest city": [np.mean]}).sort_values(by=("Population in largest city","mean"),ascending=False)
axf1 = fig.add_subplot(gs[0, 0])
box_style=dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# Text inside a box
axf1.set_title('Population in the Largest City \nby Year')
axf1.pie(np.array(PopGDP0.values).flatten(),labels=PopGDP0.index, autopct ='% 1.1f %%',shadow = True, startangle = 90)
axf1.grid()

axf2 = fig.add_subplot(gs[0, 1])
PopGDP2=PopGDP.copy()
PopGDP2=PopGDP2.set_index('Year')
axf2.set_title('Overall Inflation')
axf2.hist(PopGDP2['Inflation'], bins = 7, density=1, histtype='bar', label=PopGDP2.index,stacked=True,color="g",alpha=0.5)
axf2.set_xlabel("Inflation",color="m")
axf2.set_ylabel("Distribution",color="m")
axf2.grid()

PopGDP1=PopGDP.groupby('Year').agg({"Population ages 65 and above, male": [np.mean]}).sort_values(by=("Population ages 65 and above, male","mean"),ascending=False)
axf3 = fig.add_subplot(gs[0, 2])
axf3.set_title('Population (ages 65 and above) of Male \nby Year')
axf3.pie(np.array(PopGDP1.values).flatten(),labels=PopGDP1.index, autopct ='% 1.1f %%',shadow = True, startangle = 90)
axf3.grid()

axf4 = fig.add_subplot(gs[1, 0])
axf4.set_title('Inflation by Year')
axf4.barh(PopGDP['Year'],PopGDP['Inflation'], color=["#00FFFF","#FFA500"])
axf4.grid()

axf5 = fig.add_subplot(gs[1, 1])
axf5.set_title('Total GDP Values by Year')
axf5.scatter(PopGDP['Population in largest city'],PopGDP['GDP Total'], color="#3B9C9C",alpha=0.6)
axf5.set_xlabel("Population in largest city",color="b")
axf5.set_ylabel("GDP Total",color="b")
axf5.grid()

PopGDP1=PopGDP.copy()
PopGDP1['Year']=np.array(PopGDP1['Year'],str)
axf6 = fig.add_subplot(gs[1, 2])
axf6.set_title('Growth of GDP by Year')
axf6.barh(PopGDP1['Year'],PopGDP['GDP growth'], color=["#00FFFF","#FFA500"])
axf6.grid()

plt.figtext(0.5, 0.98, "Analyzing Popuplation with multiple indicators from 2012 to 2020 ", ha="center", va="center", fontsize=18, bbox={"facecolor":"orange", "alpha":0.5})
plt.figtext(0.99, 0.98, "Deepika Cherupally", ha="right", va="center", fontsize=18)
plt.figtext(0.1, 0.98, "22031854", ha="right", va="center", fontsize=18)
plt.figtext(0.5,0.07, "The visualization is done between different indicators and population using different plotting techniques we can compare different things.", ha="center", va="center", fontsize=8)
plt.figtext(0.5,0.05, "Pie chart1 shows largest city Population in years between 2012 to 2020, from the visualization we can get largest city poplation ratios which seems to be same in the compared years,and in Pie chart-2 it shows visualization of male at age 65 and above in significant years which is almost same in every year.", ha="center", va="center", fontsize=8)
plt.figtext(0.5,0.03, "Histogram is used to compare the population with the inflation period which is high at period of time and is reduced to zero in the mid.The Scatter Plot is used here to show the Total GDP values with Year.", ha="center", va="center", fontsize=8)
plt.figtext(0.5,0.01, "Bar Charts1 is used to show the same inflation period between 2012 - 2020 which is represent the low value in the starting period 2012 and ias gradually increased and has high value at 2017. Bar chart 2- is to show the visualization bewtween Growth of GDP by year which was high at 2017.", ha="center", va="center", fontsize=8)


#plt.savefig("22031854.png", dpi=300)
